Read data.

In [8]:
import pandas as pd
data = pd.read_csv('./HW2 附件/HW2_bike-sharing_train.csv')

Preprocessing

In [13]:

df = data.copy()
print(df.columns)
df['datetime'] = pd.to_datetime(df['datetime'])

# # Extract year, month, day, hour, minute, and second as separate columns
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
print(df)

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'count'],
      dtype='object')
                datetime  season  holiday  workingday  weather   temp   atemp  \
0    2011-01-01 00:00:00       1        0           0        1   9.84  14.395   
1    2011-01-01 01:00:00       1        0           0        1   9.02  13.635   
2    2011-01-01 02:00:00       1        0           0        1   9.02  13.635   
3    2011-01-01 03:00:00       1        0           0        1   9.84  14.395   
4    2011-01-01 04:00:00       1        0           0        1   9.84  14.395   
...                  ...     ...      ...         ...      ...    ...     ...   
8381 2012-12-19 17:00:00       4        0           1        1  16.40  20.455   
8382 2012-12-19 18:00:00       4        0           1        1  15.58  19.695   
8383 2012-12-19 19:00:00       4        0           1        1  15.58  19.695   
8384 2012-12-19 20:00:00       4        0   